In [168]:
# get the model predictions
y_pred = model.predict(X_test)

# get the top two predictions
top_two_pred = np.argsort(y_pred, axis=-1)[:, -2:]

# convert your one-hot encoded labels back to class indices
y_test_class_indices = np.argmax(y_test_encoded, axis=-1)

# calculate accuracy
correct = [y in pred for y, pred in zip(y_test_class_indices, top_two_pred)]
accuracy = np.mean(correct)

print(f'Top-2 accuracy: {accuracy * 100:.2f}%')

94/94 [==============================] - 0s 2ms/step
Top-2 accuracy: 81.60%


In [163]:
top_two_pred

array([[1, 3],
       [2, 1],
       [1, 3],
       ...,
       [1, 3],
       [3, 2],
       [1, 2]])

In [143]:
import os 
import pandas as pd
import numpy as np
folder_path = './dataset2'

### Adds headers to files

In [144]:
header = ['pick_num', 'team_name', 'player', 'player_team', 'player_pos']

for filename in os.listdir(folder_path):
    if not os.path.isdir(os.path.join(folder_path, filename)): 
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()
        lines.insert(0, ','.join(header) + '\n')
        with open(file_path, 'w') as file:
            file.writelines(lines)

### Checks if theres 12 teams in each draft

In [146]:
for filename in os.listdir(folder_path):
        if not os.path.isdir(os.path.join(folder_path, filename)):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)  
            teams = df['team_name'].unique()
            if len(teams) != 12:
                print(f"File '{filename}': Number of unique teams does not equal 12.")

### Encode team names

In [147]:

for filename in os.listdir(folder_path):
    if not os.path.isdir(os.path.join(folder_path, filename)):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)  
        teams = df['team_name'].unique()
        team_mapping = {team: f'Team{i+1}' for i, team in enumerate(teams)}
        df['team_name'] = df['team_name'].map(team_mapping)
        df.to_csv(file_path, index=False) 


### Checks if pick order in each draft is correct

In [1]:
for filename in os.listdir(folder_path):
    if not os.path.isdir(os.path.join(folder_path, filename)):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path) 
        expected_draft_order = (list(range(1, 13)) + list(range(12, 0, -1)))*15
        expected_draft_order = expected_draft_order[:int(len(expected_draft_order)/2)]
        draft_order_in_file = [int(''.join(filter(str.isdigit, element))) for element in df['team_name']]
        if expected_draft_order != draft_order_in_file:
                print(f"File '{filename}': Draft order incorrect")

NameError: name 'os' is not defined

### Adding ADP values to each draft 

In [154]:
adp_df = pd.read_csv('FantasyPros_2023_Overall_ADP_Rankings.csv')
adp_df = adp_df.rename(columns={'AVG': 'ADP', 'Player' : 'player'})
adp_df = adp_df[['player','ADP']]
adp_df = adp_df.dropna(how='all')

#high default ADP value for Defense and Kickers
default_adp = adp_df['ADP'].max() + 10

for filename in os.listdir(folder_path):
    if not os.path.isdir(os.path.join(folder_path, filename)):
        file_path = os.path.join(folder_path, filename)
        draft_df = pd.read_csv(file_path)
        draft_df = draft_df.merge(adp_df, on='player', how='left')
        # draft_df['ADP'] = draft_df['ADP'].fillna(np.where(draft_df['player_pos'].isin(['DST', 'K']), default_adp, np.nan))
        missing_adp_players = draft_df[draft_df['ADP'].isnull()]['player'].unique()
        # missing_adp_players = []

        for i in range(len(draft_df)):
                # If the player's ADP is missing, add to the list and assign default_adp
                if pd.isnull(draft_df.loc[i, 'ADP']):
                    missing_adp_players.append(draft_df.loc[i, 'player'])
                    draft_df.loc[i, 'ADP'] = default_adp
                    
        if len(missing_adp_players) > 0:
            print(f"In file {filename}, these players have missing ADP values: {missing_adp_players}")

        # Save the dataframe back to the file
        draft_df.to_csv(file_path, index=False)

KeyError: 'ADP'

### Create State Representations (Model Input)

In [155]:
default_adp = adp_df['ADP'].max() + 10

#Representation 1: Accounts for players left on the board

#Returns 4 element lists for each position. First element is the number of players at that position
#that haven't been drafted. Next 3 elements are smallest ADP values of undrafted players at that position
def get_remaining_players_repr(df, current_pick_num):
    remaining_players = df[df['pick_num'] > current_pick_num]
    remaining_players = remaining_players.sort_values('ADP')

    positions = ['QB', 'RB', 'WR', 'TE', 'DST', 'K']

    remaining_repr = {}
    for pos in positions:
        pos_players = remaining_players[remaining_players['player_pos'] == pos]
        pos_count = len(pos_players)
        pos_adp_values = pos_players['ADP'].nsmallest(3).tolist()

        # If less than 3 players, pad w/ default_adp (max + 10)
        while len(pos_adp_values) < 3:
            pos_adp_values.append(default_adp)

        remaining_repr[pos] = [pos_count] + pos_adp_values
    ret_df = pd.DataFrame(remaining_repr).T
    return ret_df.values

#Representation 2: Accounts for current roster of the team picking 

#Returns list of team's roster indicating position slots that can be filled 
# 0 = Slot filled 
def get_team_roster_repr(df, team_name, current_pick_num):
    df = df[df['pick_num'] < current_pick_num]
    team_picks = df[df['team_name'] == team_name]
    position_counts = pd.get_dummies(team_picks['player_pos']).sum()

    positions = ['QB', 'RB', 'WR', 'TE', 'DST', 'K']
    max_players = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'DST': 1, 'K': 1}
    flex = 1  # flex position
    bench = 6

    roster_repr = []
    for pos in positions:
        if pos in position_counts:
            roster_repr.append(max_players[pos] - position_counts[pos])
        else:
            roster_repr.append(max_players[pos])

    # Handle flex position
    if 'RB' in position_counts:
        rb_extra = max(0, position_counts['RB'] - max_players['RB'])
        if rb_extra > 0:
            flex -= 1
            bench -= rb_extra - 1
    if 'WR' in position_counts and flex > 0:
        wr_extra = max(0, position_counts['WR'] - max_players['WR'])
        if wr_extra > 0:
            flex -= 1
            bench -= wr_extra - 1

    roster_repr.append(flex)
    bench -= sum(max(0, position_counts.get(pos, 0) - max_players[pos]) for pos in positions)
    roster_repr.append(bench)
    return np.array(roster_repr)

#Forms a single state representation based on the two representations 
def get_state_representation(df, current_pick_num, team_name, max_players=180):
    team_roster_repr = get_team_roster_repr(df, team_name, current_pick_num)
    remaining_players_repr = get_remaining_players_repr(df, current_pick_num)
    state_repr = np.concatenate([team_roster_repr, remaining_players_repr], axis=None)
    return state_repr

### Create Input/Output Pairs to Train Model 

In [156]:
inputs = []
outputs = []

expected_draft_order = (list(range(1, 13)) + list(range(12, 0, -1)))*15
expected_draft_order = expected_draft_order[:int(len(expected_draft_order)/2)]

for filename in os.listdir(folder_path):
    if not os.path.isdir(os.path.join(folder_path, filename)):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        for pick_num in range(1, df['pick_num'].max()):
            # generate the state representation for the current pick
            teamID = expected_draft_order[pick_num - 1]
            state_repr = get_state_representation(df, pick_num, f'Team{teamID}')  # replace team_name with actual team name
            # get the position of the player picked next
            next_pick_pos = df.loc[df['pick_num'] == pick_num+1, 'player_pos'].values[0]
            # store the input-output pair
            inputs.append(state_repr)
            outputs.append(next_pick_pos)        

In [157]:
len(inputs)
len(outputs)

6086

In [158]:

for input_val, output_val in zip(inputs, outputs):
    if output_val not in ["K", "DST"]:
        filtered_inputs.append(input_val)
        filtered_outputs.append(output_val)

inputs_f = np.array(filtered_inputs)
outputs_f = np.array(filtered_outputs)

In [159]:
len(inputs_f)
len(outputs_f)

9926

### Split Data

In [160]:
from sklearn.model_selection import train_test_split

inputs = np.array(inputs)
outputs = np.array(outputs)

X_train, X_test, y_train, y_test = train_test_split(inputs_f, outputs_f, test_size=0.3, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [161]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_val_encoded = encoder.transform(y_val)
y_test_encoded = encoder.transform(y_test)
y_train_encoded = to_categorical(y_train_encoded)
y_val_encoded = to_categorical(y_val_encoded)
y_test_encoded = to_categorical(y_test_encoded)

model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(y_train_encoded.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train_encoded, epochs=100, validation_data=(X_val, y_val_encoded))

loss, accuracy = model.evaluate(X_test, y_test_encoded)

print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')


2023-05-27 19:37:53.481459: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-27 19:37:53.483734: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-27 19:37:53.487060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/100


2023-05-27 19:37:54.526439: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-27 19:37:54.532966: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-27 19:37:54.536195: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

171/174 [============================>.] - ETA: 0s - loss: 1.1860 - accuracy: 0.4388

2023-05-27 19:37:59.355212: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-27 19:37:59.358460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-27 19:37:59.361962: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

174/174 [==============================] - 6s 9ms/step - loss: 1.1872 - accuracy: 0.4383 - val_loss: 1.1622 - val_accuracy: 0.4662
Epoch 2/100
174/174 [==============================] - 1s 4ms/step - loss: 1.1705 - accuracy: 0.4547 - val_loss: 1.1538 - val_accuracy: 0.4568
Epoch 3/100
174/174 [==============================] - 1s 4ms/step - loss: 1.1623 - accuracy: 0.4626 - val_loss: 1.1459 - val_accuracy: 0.4647
Epoch 4/100
174/174 [==============================] - 1s 4ms/step - loss: 1.1633 - accuracy: 0.4604 - val_loss: 1.1410 - val_accuracy: 0.4748
Epoch 5/100
174/174 [==============================] - 1s 3ms/step - loss: 1.1591 - accuracy: 0.4622 - val_loss: 1.1451 - val_accuracy: 0.4676
Epoch 6/100
174/174 [==============================] - 1s 3ms/step - loss: 1.1593 - accuracy: 0.4566 - val_loss: 1.1464 - val_accuracy: 0.4755
Epoch 7/100
174/174 [==============================] - 1s 4ms/step - loss: 1.1514 - accuracy: 0.4674 - val_loss: 1.1333 - val_accuracy: 0.4676
Epoch 8/100

In [167]:
df = pd.read_csv('./dataset2/draft_9HLHrxhB')
current_pick_num = 31
# pass in your current pick number
team_name = 'Team7'
max_players = 180  # or set it to your required number


next_pick_pos = df.loc[df['pick_num'] == current_pick_num+1, 'player_pos'].values[0]

# Generate the input vector for your model
input_vector = get_state_representation(df, current_pick_num, team_name, max_players)

# Reshape this to match the input shape that your model expects
input_vector = input_vector.reshape(1, 1, -1)

# Make the prediction
y_pred = model.predict(input_vector)

# Get the top two predicted class indices
top_two_indices = np.argsort(y_pred[0])[-2:]

# Convert these indices back to original labels
top_two_predictions = encoder.inverse_transform(top_two_indices)

# Get the corresponding probabilities
top_two_probabilities = y_pred[0][top_two_indices]

# Output the results
for pred, prob in zip(top_two_predictions, top_two_probabilities):
    print(f'Predicted label: {pred}, Probability: {prob}')



1/1 [==============================] - 0s 53ms/step
Predicted label: RB, Probability: 0.35127750039100647
Predicted label: WR, Probability: 0.4430779218673706


In [1]:
import os 
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2023-05-28 20:07:01.463429: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:

def preprocess_data(data_folders):
    inputs = []
    outputs = []
    inputs_f = []
    outputs_f = []

    expected_draft_order = (list(range(1, 13)) + list(range(12, 0, -1)))*15
    expected_draft_order = expected_draft_order[:int(len(expected_draft_order)/2)]

    for folder_path in data_folders:
        for filename in os.listdir(folder_path):
            if not os.path.isdir(os.path.join(folder_path, filename)):
                file_path = os.path.join(folder_path, filename)
                best_teams = get_best_teams(file_path)
                df = pd.read_csv(file_path)
                
                for pick_num in range(1, df['pick_num'].max()):
                    # generate the state representation for the current pick
                    teamID = expected_draft_order[pick_num - 1]
                    state_repr = get_state_representation(df, pick_num, f'Team{teamID}')  # replace team_name with actual team name
                    # get the position of the player picked next
                    next_pick_pos = df.loc[df['pick_num'] == pick_num+1, 'player_pos'].values[0]
                    # store the input-output pair
                    inputs.append(state_repr)
                    outputs.append(next_pick_pos)
        
        for input_val, output_val in zip(inputs, outputs):
            if output_val not in ["K", "DST"]:
                inputs_f.append(input_val)
                outputs_f.append(output_val)

    inputs_f = np.array(inputs_f)
    outputs_f = np.array(outputs_f)

    return inputs_f,outputs_f


def get_best_teams(file_path):

SyntaxError: incomplete input (3921344105.py, line 38)

In [52]:
df = pd.read_csv('./dataset2/draft_0paljEN7')
df = df[df['player_pos'] != 'K']
df = df[df['player_pos'] != 'DST']

draft_scores = {}

grouped_teams = df.groupby('team_name')

for group_name, group_indicies in grouped_teams.groups.items():

    positions = {'QB': 0, 'RB': 0, 'WR': 0, 'TE': 0}

    starting_lineup = []
    bench = []

    team_df = df.loc[group_indicies]
    tean_df = df.sort_values('ADP')

    for index,row in team_df.iterrows():
        pos = row['player_pos']

        if ((pos in ['RB', 'WR'] and positions[pos] < 2) or
            (pos in ['QB', 'TE'] and positions[pos] < 1)):

            starting_lineup.append(row['ADP'])
            positions[pos] += 1
        #flex position case
        elif pos in ['RB', 'WR'] and sum([value for key, value in positions.items() if key in ['RB', 'WR']]) < 3:
            starting_lineup.append(row['ADP'])
            positions[pos] += 1

        else:
            bench.append(row['ADP'])
    
    draft_scores[str(group_name)] =  sum([i*1.5 for i in starting_lineup]) + sum(bench)
        

    



In [46]:
df = pd.read_csv('./dataset2/draft_0paljEN7')

df = df[df['player_pos'] != 'K']
df = df[df['player_pos'] != 'DST']
df


,pick_num,team_name,player,player_team,player_pos,ADP
0,1,Team1,Justin Jefferson,MIN,WR,1.0
1,2,Team2,Ja'Marr Chase,CIN,WR,2.0
2,3,Team3,Christian McCaffrey,SF,RB,3.0
3,4,Team4,Austin Ekeler,LAC,RB,10.0
4,5,Team5,Cooper Kupp,LAR,WR,5.0
...,...,...,...,...,...,...
151,152,Team8,Michael Carter,NYJ,RB,222.5
152,153,Team9,Wan'Dale Robinson,NYG,WR,176.5
153,154,Team10,Greg Dulcich,DEN,TE,150.5
154,155,Team11,Cordarrelle Patterson,ATL,RB,238.0


In [53]:
df = df[df['team_name'] == 'Team2']
df

,pick_num,team_name,player,player_team,player_pos,ADP
1,2,Team2,Ja'Marr Chase,CIN,WR,2.0
22,23,Team2,Amon-Ra St. Brown,DET,WR,13.5
25,26,Team2,Patrick Mahomes II,KC,QB,18.5
46,47,Team2,Kenneth Walker III,SEA,RB,46.0
49,50,Team2,Jerry Jeudy,DEN,WR,46.5
70,71,Team2,Darren Waller,NYG,TE,79.0
73,74,Team2,Alexander Mattison,MIN,RB,90.0
94,95,Team2,Zach Charbonnet,SEA,RB,99.5
97,98,Team2,Quentin Johnston,LAC,WR,86.5
118,119,Team2,Elijah Mitchell,SF,RB,127.5


In [48]:

df = pd.DataFrame.from_dict(draft_scores, orient='index').reset_index()
df.columns = ['team_name', 'score']
df = df.sort_values(by='score', ascending=True)
df

,team_name,score
5,Team3,1090.25
0,Team1,1121.75
4,Team2,1125.00
8,Team6,1125.00
7,Team5,1170.50
11,Team9,1196.75
1,Team10,1215.25
9,Team7,1234.00
6,Team4,1242.25
3,Team12,1334.50


In [67]:
df = pd.read_csv('./dataset2/draft_1Qac1wKR')
def get_best_teams(df):
    df = df[df['player_pos'] != 'K']
    df = df[df['player_pos'] != 'DST']

    draft_scores = {}

    grouped_teams = df.groupby('team_name')

    for group_name, group_indicies in grouped_teams.groups.items():

        positions = {'QB': 0, 'RB': 0, 'WR': 0, 'TE': 0}

        starting_lineup = []
        bench = []

        team_df = df.loc[group_indicies]
        tean_df = df.sort_values('ADP')

        for index,row in team_df.iterrows():
            pos = row['player_pos']

            if ((pos in ['RB', 'WR'] and positions[pos] < 2) or
                (pos in ['QB', 'TE'] and positions[pos] < 1)):

                starting_lineup.append(row['ADP'])
                positions[pos] += 1
            #flex position case
            elif pos in ['RB', 'WR'] and sum([value for key, value in positions.items() if key in ['RB', 'WR']]) < 3:
                starting_lineup.append(row['ADP'])
                positions[pos] += 1

            else:
                bench.append(row['ADP'])
        
        draft_scores[str(group_name)] =  sum([i*1.5 for i in starting_lineup]) + sum(bench)

    draft_score_df = pd.DataFrame.from_dict(draft_scores, orient='index').reset_index()
    draft_score_df.columns = ['team_name', 'score']
    draft_score_df = draft_score_df.sort_values(by='score', ascending=True)

    return list(draft_score_df[:4]['team_name'])

print(get_best_teams(df))

['Team3', 'Team5', 'Team2', 'Team1']


In [66]:

df = pd.DataFrame.from_dict(draft_scores, orient='index').reset_index()
df.columns = ['team_name', 'score']
df = df.sort_values(by='score', ascending=True)

x = list(df[:4]['team_name'])

df2 = pd.read_csv('./dataset2/draft_1Qac1wKR')


df2 = df[df['team_name'].isin(list(df[:4]['team_name']))]


,pick_num,team_name,player,player_team,player_pos,ADP
0,1,Team1,Christian McCaffrey,SF,RB,3.0
1,2,Team2,Justin Jefferson,MIN,WR,1.0
2,3,Team3,Ja'Marr Chase,CIN,WR,2.0
3,4,Team4,Austin Ekeler,LAC,RB,10.0
4,5,Team5,Jonathan Taylor,IND,RB,13.5
...,...,...,...,...,...,...
175,176,Team8,Jake Moody,SF,K,438.0
176,177,Team9,Brandon McManus,JAC,K,438.0
177,178,Team10,Jason Sanders,MIA,K,438.0
178,179,Team11,Matt Gay,IND,K,438.0
